# Roll NO : M22AIE236

In [ ]:
#version : 1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt

# Step 1: Load STL10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.STL10(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.STL10(root='./data', split='test', download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Step 2: Load pre-trained ResNet101 model
resnet101 = models.resnet101(pretrained=True)
num_ftrs = resnet101.fc.in_features
resnet101.fc = nn.Linear(num_ftrs, 10)  # Replace final fully connected layer for 10 classes

# Step 3: Define optimizers
optimizers = {
    'Adam': optim.Adam(resnet101.parameters(), lr=0.001),
    'Adagrad': optim.Adagrad(resnet101.parameters(), lr=0.001),
    'Adadelta': optim.Adadelta(resnet101.parameters(), lr=1.0)
}

# Step 4: Train the model with different optimizers
num_epochs = 10
criterion = nn.CrossEntropyLoss()
train_losses = {optimizer_name: [] for optimizer_name in optimizers}
train_accuracies = {optimizer_name: [] for optimizer_name in optimizers}

for optimizer_name, optimizer in optimizers.items():
    print(f"Training with optimizer: {optimizer_name}")
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = resnet101(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100.0 * correct / total

        train_losses[optimizer_name].append(epoch_loss)
        train_accuracies[optimizer_name].append(epoch_accuracy)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Step 5: Plot training loss and accuracy curves
plt.figure(figsize=(12, 6))
for optimizer_name, losses in train_losses.items():
    plt.plot(losses, label=f'{optimizer_name} Loss')

plt.title('Training Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
for optimizer_name, accuracies in train_accuracies.items():
    plt.plot(accuracies, label=f'{optimizer_name} Accuracy')

plt.title('Training Accuracy Curves')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()

# Step 6: Report final top-5 test accuracy
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = resnet101(images)
        _, predicted = torch.topk(outputs, 5, dim=1)  # Get top-5 predictions
        total += labels.size(0)
        correct += sum([1 for i in range(labels.size(0)) if labels[i] in predicted[i]])

    print(f"Final Top-5 Test Accuracy: {100.0 * correct / total:.2f}%")


100%|██████████| 2640397119/2640397119 [04:01<00:00, 10935751.10it/s]


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 125MB/s]


Training with optimizer: Adam
